In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

class INFOGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.num_classes = 10
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 72


        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', self.mutual_info_loss]

        # Build and the discriminator and recognition network
        self.discriminator, self.auxilliary = self.build_disk_and_q_net()

        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the recognition network Q
        self.auxilliary.compile(loss=[self.mutual_info_loss],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        gen_input = Input(shape=(self.latent_dim,))
        img = self.generator(gen_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        valid = self.discriminator(img)
        # The recognition network produces the label
        target_label = self.auxilliary(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(gen_input, [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        gen_input = Input(shape=(self.latent_dim,))
        img = model(gen_input)

        model.summary()

        return Model(gen_input, img)


    def build_disk_and_q_net(self):

        img = Input(shape=self.img_shape)

        # Shared layers between discriminator and recognition network
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Flatten())

        img_embedding = model(img)

        # Discriminator
        validity = Dense(1, activation='sigmoid')(img_embedding)

        # Recognition
        q_net = Dense(128, activation='relu')(img_embedding)
        label = Dense(self.num_classes, activation='softmax')(q_net)

        # Return discriminator and recognition network
        return Model(img, validity), Model(img, label)


    def mutual_info_loss(self, c, c_given_x):
        """The mutual information metric we aim to minimize"""
        eps = 1e-8
        conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
        entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

        return conditional_entropy + entropy

    def sample_generator_input(self, batch_size):
        # Generator inputs
        sampled_noise = np.random.normal(0, 1, (batch_size, 62))
        sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)
        sampled_labels = to_categorical(sampled_labels, num_classes=self.num_classes)

        return sampled_noise, sampled_labels

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and categorical labels
            sampled_noise, sampled_labels = self.sample_generator_input(batch_size)
            gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(gen_input)

            # Train on real and generated data
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)

            # Avg. loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator and Q-network
            # ---------------------

            g_loss = self.combined.train_on_batch(gen_input, [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 10

        fig, axs = plt.subplots(r, c)
        for i in range(c):
            sampled_noise, _ = self.sample_generator_input(c)
            label = to_categorical(np.full(fill_value=i, shape=(r,1)), num_classes=self.num_classes)
            gen_input = np.concatenate((sampled_noise, label), axis=1)
            gen_imgs = self.generator.predict(gen_input)
            gen_imgs = 0.5 * gen_imgs + 0.5
            for j in range(r):
                axs[j,i].imshow(gen_imgs[j,:,:,0], cmap='gray')
                axs[j,i].axis('off')
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")


if __name__ == '__main__':
    infogan = INFOGAN()
    infogan.train(epochs=50000, batch_size=128, sample_interval=50)


C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              457856    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 128)       512       
__________

C:\Users\adward\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.13, acc.: 34.77%] [Q loss: 0.83] [G loss: 2.99]
1 [D loss: 0.79, acc.: 63.67%] [Q loss: 1.13] [G loss: 3.02]
2 [D loss: 0.69, acc.: 65.23%] [Q loss: 1.30] [G loss: 3.03]
3 [D loss: 0.59, acc.: 68.36%] [Q loss: 1.75] [G loss: 2.85]
4 [D loss: 0.56, acc.: 69.53%] [Q loss: 1.59] [G loss: 2.71]
5 [D loss: 0.57, acc.: 67.58%] [Q loss: 1.63] [G loss: 2.59]
6 [D loss: 0.62, acc.: 65.23%] [Q loss: 1.76] [G loss: 2.52]
7 [D loss: 0.63, acc.: 67.97%] [Q loss: 1.82] [G loss: 2.69]
8 [D loss: 0.56, acc.: 69.14%] [Q loss: 1.80] [G loss: 2.56]
9 [D loss: 0.63, acc.: 65.23%] [Q loss: 1.60] [G loss: 2.67]
10 [D loss: 0.60, acc.: 66.80%] [Q loss: 1.70] [G loss: 2.66]
11 [D loss: 0.58, acc.: 67.19%] [Q loss: 1.63] [G loss: 2.68]
12 [D loss: 0.52, acc.: 71.48%] [Q loss: 1.64] [G loss: 2.67]
13 [D loss: 0.49, acc.: 70.70%] [Q loss: 1.79] [G loss: 2.65]
14 [D loss: 0.46, acc.: 71.88%] [Q loss: 1.65] [G loss: 2.44]
15 [D loss: 0.55, acc.: 67.97%] [Q loss: 1.66] [G loss: 2.58]
16 [D loss: 0.50, 

132 [D loss: 0.46, acc.: 76.17%] [Q loss: 2.08] [G loss: 0.26]
133 [D loss: 0.44, acc.: 74.22%] [Q loss: 2.29] [G loss: 0.17]
134 [D loss: 0.53, acc.: 69.92%] [Q loss: 2.28] [G loss: 0.26]
135 [D loss: 0.45, acc.: 76.56%] [Q loss: 2.04] [G loss: 0.23]
136 [D loss: 0.52, acc.: 69.92%] [Q loss: 2.14] [G loss: 0.25]
137 [D loss: 0.47, acc.: 73.83%] [Q loss: 2.01] [G loss: 0.19]
138 [D loss: 0.48, acc.: 73.83%] [Q loss: 2.14] [G loss: 0.22]
139 [D loss: 0.53, acc.: 69.14%] [Q loss: 2.11] [G loss: 0.33]
140 [D loss: 0.44, acc.: 78.12%] [Q loss: 2.07] [G loss: 0.18]
141 [D loss: 0.55, acc.: 69.14%] [Q loss: 2.02] [G loss: 0.23]
142 [D loss: 0.50, acc.: 71.88%] [Q loss: 2.03] [G loss: 0.20]
143 [D loss: 0.53, acc.: 70.31%] [Q loss: 2.07] [G loss: 0.18]
144 [D loss: 0.48, acc.: 74.22%] [Q loss: 1.98] [G loss: 0.21]
145 [D loss: 0.53, acc.: 69.53%] [Q loss: 2.00] [G loss: 0.29]
146 [D loss: 0.57, acc.: 69.92%] [Q loss: 2.09] [G loss: 0.19]
147 [D loss: 0.56, acc.: 69.92%] [Q loss: 2.16] [G loss

263 [D loss: 0.69, acc.: 59.77%] [Q loss: 1.22] [G loss: 0.07]
264 [D loss: 0.72, acc.: 58.98%] [Q loss: 1.19] [G loss: 0.06]
265 [D loss: 0.69, acc.: 60.16%] [Q loss: 1.28] [G loss: 0.06]
266 [D loss: 0.73, acc.: 56.64%] [Q loss: 1.17] [G loss: 0.13]
267 [D loss: 0.63, acc.: 64.84%] [Q loss: 1.26] [G loss: 0.11]
268 [D loss: 0.71, acc.: 58.59%] [Q loss: 1.15] [G loss: 0.05]
269 [D loss: 0.67, acc.: 62.89%] [Q loss: 1.16] [G loss: 0.13]
270 [D loss: 0.70, acc.: 60.55%] [Q loss: 1.15] [G loss: 0.06]
271 [D loss: 0.75, acc.: 57.03%] [Q loss: 1.16] [G loss: 0.09]
272 [D loss: 0.67, acc.: 61.72%] [Q loss: 1.18] [G loss: 0.05]
273 [D loss: 0.73, acc.: 60.16%] [Q loss: 1.15] [G loss: 0.10]
274 [D loss: 0.76, acc.: 54.69%] [Q loss: 1.16] [G loss: 0.11]
275 [D loss: 0.69, acc.: 60.55%] [Q loss: 1.16] [G loss: 0.12]
276 [D loss: 0.74, acc.: 57.81%] [Q loss: 1.24] [G loss: 0.05]
277 [D loss: 0.74, acc.: 59.38%] [Q loss: 1.21] [G loss: 0.08]
278 [D loss: 0.66, acc.: 60.94%] [Q loss: 1.15] [G loss

394 [D loss: 0.66, acc.: 60.94%] [Q loss: 1.03] [G loss: 0.12]
395 [D loss: 0.64, acc.: 61.72%] [Q loss: 1.07] [G loss: 0.06]
396 [D loss: 0.75, acc.: 58.98%] [Q loss: 1.03] [G loss: 0.04]
397 [D loss: 0.72, acc.: 58.59%] [Q loss: 0.91] [G loss: 0.08]
398 [D loss: 0.67, acc.: 57.03%] [Q loss: 1.05] [G loss: 0.03]
399 [D loss: 0.66, acc.: 58.59%] [Q loss: 1.07] [G loss: 0.03]
400 [D loss: 0.65, acc.: 62.89%] [Q loss: 1.08] [G loss: 0.02]
401 [D loss: 0.66, acc.: 58.59%] [Q loss: 1.07] [G loss: 0.08]
402 [D loss: 0.65, acc.: 61.33%] [Q loss: 1.07] [G loss: 0.03]
403 [D loss: 0.71, acc.: 55.47%] [Q loss: 1.12] [G loss: 0.04]
404 [D loss: 0.68, acc.: 60.55%] [Q loss: 1.04] [G loss: 0.06]
405 [D loss: 0.69, acc.: 61.33%] [Q loss: 1.02] [G loss: 0.05]
406 [D loss: 0.71, acc.: 57.03%] [Q loss: 1.02] [G loss: 0.06]
407 [D loss: 0.74, acc.: 60.16%] [Q loss: 0.97] [G loss: 0.09]
408 [D loss: 0.76, acc.: 58.20%] [Q loss: 0.99] [G loss: 0.09]
409 [D loss: 0.72, acc.: 57.03%] [Q loss: 1.04] [G loss

KeyboardInterrupt: 